## Finance Scape

In [1]:
# Modules
from selenium import webdriver 
from selenium.webdriver.common.by import By 
# from selenium.webdriver.support.ui import WebDriverWait 
# from selenium.webdriver.support import expected_conditions as EC 
# from selenium.common.exceptions import TimeoutException
from selenium import webdriver
import chromedriver_binary  # Adds chromedriver binary to path
from selenium.webdriver.chrome.options import Options
from coinbase.wallet.client import Client

# Log-in Credentials
from Credentials import *

# Formatting
from money import Money
import smtplib
from email.message import EmailMessage
import time



class Scraper:
    
    def __init__(self,credentials):
        self.credentials = credentials
        
    # Money Dashboard
    def MD(self):
        chrome_options = Options()  
        chrome_options.add_argument("--headless")  
        driver = webdriver.Chrome(options=chrome_options)
        driver.get('https://my.moneydashboard.com/?signin=true')

        time.sleep(1)
        s=driver.find_element(By.XPATH, "/html/body/div[4]/md-dialog/md-dialog-actions/button")
        s.click()
        time.sleep(1)

        e=driver.find_element(By.NAME, "email")
        e.send_keys(credentials["md_email"])
        p=driver.find_element(By.NAME, "password")
        p.send_keys(credentials["md_password"])

        s=driver.find_element(By.XPATH, "/html/body/ui-view/div[2]/div[1]/div/form/div/div[1]/div[2]/button")
        s.click()
        time.sleep(1)
        s=driver.find_element(By.XPATH, '//*[@id="navbar"]/div[2]/div[1]/md-icon')
        s.click()
        time.sleep(1)
        value = driver.find_elements_by_xpath('//*[@id="navbar"]/div[3]/div[1]/div[1]/div[1]')[0].text
        return value

    
    # Hargreaves Lansdown
    def HL(self):
        chrome_options = Options()  
        chrome_options.add_argument("--headless")  
        driver = webdriver.Chrome(options=chrome_options)
        driver.get('https://online.hl.co.uk/my-accounts/login-step-one')


        time.sleep(1)

        e=driver.find_element(By.XPATH, '//*[@id="username"]')
        e.send_keys(credentials["HL_username"])
        p=driver.find_element(By.XPATH, '//*[@id="date-of-birth"]')
        p.send_keys(credentials["HL_dob"])
        s=driver.find_element(By.XPATH, '//*[@id="login"]/p[1]/input')
        s.click()

        time.sleep(1)

        e=driver.find_element(By.XPATH, '//*[@id="online-password-verification"]')
        e.send_keys(credentials["HL_password"])

        secure_number_input1 = driver.find_element_by_name('secure-number[1]')
        secure_number_input2 = driver.find_element_by_name('secure-number[2]')
        secure_number_input3 = driver.find_element_by_name('secure-number[3]')

        val1=int(secure_number_input1.get_attribute("title")[10])-1
        val2=int(secure_number_input2.get_attribute("title")[10])-1
        val3=int(secure_number_input3.get_attribute("title")[10])-1

        secure_number_input1.send_keys(credentials["HL_secure_number"][val1])
        secure_number_input2.send_keys(credentials["HL_secure_number"][val2])
        secure_number_input3.send_keys(credentials["HL_secure_number"][val3])


        s=driver.find_element(By.XPATH, '//*[@id="submit"]')
        s.click()

        value2 = driver.find_elements_by_xpath('//*[@id="content-body-full"]/div/div[2]/table/tfoot/tr/td[2]')[0].text
        return value2

    
    def CB(self):
        client = Client(credentials["C_api_key"], credentials["C_api_secret"])
        total = 0
        message = []
        accounts = client.get_accounts()
        for wallet in accounts.data:
            message.append( str(wallet['name']) + ' ' +   str(wallet['native_balance']) )
            value = str( wallet['native_balance']).replace('GBP','')
            total += float(value)
        message.append( 'Total Balance: ' + 'GBP ' + str(total) )
        #print('\n'.join( message ))
        return total
    
    def email(self,text):
        gmail_user = credentials["Gmail"]
        gmail_password = credentials["Gpassword"]
        
        try:
            server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
            server.ehlo()
            server.login(gmail_user, gmail_password)
            msg = EmailMessage()
            msg.set_content(text)
            msg['Subject'] = f'Personal Finance'
            msg['From'] = gmail_user
            msg['To'] = credentials["Gto"]
            server.send_message(msg)
            server.quit()
        
        except:
            print('Something went wrong...')

    def formatting(self,md,hl,cb):
        md1 = Money(amount=md.replace(',', '')[1:], currency='GBP')
        hl1 = Money(amount=hl.replace(',', '')[1:], currency='GBP')
        cb1 = Money(amount=cb, currency='GBP')
        total = (md1+hl1+cb1)
        print(md1)
        print(hl1)
        print(cb1)
        print(total)
        text = "Santander Accounts = "+ md1.format('en_US', '¤#,##0.00')     \
                +"\nHargreaves Lansdown = "+ hl1.format('en_US', '¤#,##0.00') \
                +"\nCoinbase = "+ cb1.format('en_US', '¤#,##0.00')             \
                +"\n\nTotal = "+total.format('en_US', '¤#,##0.00')

        return text
            
    def run_code(self):
        hargreaves_lansdown = self.HL()
        money_dashboard = self.MD()
        coinbase = self.CB()
        
        body=self.formatting(money_dashboard,hargreaves_lansdown,coinbase)
        self.email(body)
        
    

instance=Scraper(credentials)
instance.run_code()

GBP 6,780.08
GBP 1,144.67
GBP 801.22
GBP 8,725.97


In [3]:
import sys
sys.executable

'C:\\Users\\Keir\\anaconda3\\python.exe'